In [1]:
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from transformers import AutoTokenizer
import torch
import requests
from urllib.parse import urljoin
from urllib.parse import urlencode
import os, re, io, time, json, html
import requests
from pathlib import Path
from urllib.parse import urljoin
from PIL import Image, UnidentifiedImageError

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import xml.etree.ElementTree as ET
from typing import List, Dict, Optional


import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

CUDA available: True
Device: NVIDIA GeForce RTX 2070 SUPER


# PubMed



In [3]:
pmcids = ["PMC9866905","PMC12449471","PMC7037937","PMC7411343","PMC12449476"]

In [4]:

pmcid = "PMC9866905"
url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
params = {"db": "pmc", "id": pmcid, "retmode": "xml"}
r = requests.get(url, params=params)
fulltext_xml = r.text


In [45]:
print(fulltext_xml)

<?xml version="1.0"  ?><!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd"><pmc-articleset><article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" xml:lang="en" article-type="research-article" dtd-version="1.4"><processing-meta base-tagset="archiving" mathml-version="3.0" table-model="xhtml" tagset-family="jats"><restricted-by>pmc</restricted-by></processing-meta><front><journal-meta><journal-id journal-id-type="nlm-ta">Pharmaceuticals (Basel)</journal-id><journal-id journal-id-type="iso-abbrev">Pharmaceuticals (Basel)</journal-id><journal-id journal-id-type="pmc-domain-id">2102</journal-id><journal-id journal-id-type="pmc-domain">pharmaceuticals</journal-id><journal-id journal-id-type="publisher-id">pharmaceuticals</journal-id><journal-title-group><journal-title>Pharmaceuticals</journal-title></journal-title-group><issn pub-type="epub">1424-8247</issn><pu

# Parsiranjea XML format

In [6]:
XLINK = "{http://www.w3.org/1999/xlink}"
WS_RE = re.compile(r"\s+")

def norm_ws(s: str) -> str:
    return WS_RE.sub(" ", (s or "")).strip()

# skip links/tables/formulas; DO NOT skip <fig> here, we handle it explicitly
SKIP_TAGS = {"ext-link", "uri", "inline-formula", "disp-formula", "table-wrap"}
INLINE_KEEP_TAGS = {"italic", "bold", "sup", "sub"}

def _fig_token(fig_el: ET.Element) -> str:
    """Build a stable placeholder for a <fig> element."""
    fid = fig_el.get("id")
    if not fid:
        g = fig_el.find(".//graphic")
        if g is not None:
            href = g.get(f"{XLINK}href") or ""
            fid = (href.rsplit("/", 1)[-1] or href).rsplit(".", 1)[0]
    if not fid:
        lbl = (fig_el.findtext("label") or "").strip() or "fig"
        fid = lbl
    return f"[[FIG::{fid}]]"

def extract_text_tokenizing_inline_figs(node: ET.Element) -> str:
    parts = []
    def walk(el: ET.Element):
        tag = el.tag

        if tag in {"ext-link", "uri", "inline-formula", "disp-formula", "table-wrap"}:
            if el.tail:
                parts.append(el.tail)
            return

        if tag == "fig":
            parts.append(_fig_token(el))          # single token where the figure sits
            if el.tail:
                parts.append(el.tail)
            return

        if tag == "xref" and (el.get("ref-type") or "").lower() == "fig":
            # do NOT tokenize mentions like "Figure 2A"; just keep flow
            if el.tail:
                parts.append(el.tail)
            return

        if el.text:
            parts.append(el.text)
        for child in list(el):
            walk(child)
        if el.tail:
            parts.append(el.tail)

    walk(node)
    return norm_ws("".join(parts))

def first(el: Optional[ET.Element], path: str) -> Optional[ET.Element]:
    return el.find(path) if el is not None else None

def section_title(sec: ET.Element) -> str:
    t = first(sec, "title")
    return extract_text_tokenizing_inline_figs(t) if t is not None else ""

def collect_sections(sec: ET.Element, parent_label: str = "") -> List[Dict]:
    chunks = []
    sid = sec.get("id")
    title_el = sec.find("title")
    title = extract_text_tokenizing_inline_figs(title_el) if title_el is not None else ""
    label = title or parent_label

    body_parts = []
    for child in list(sec):
        if child.tag == "title":
            if child.tail:
                body_parts.append(child.tail)
            continue
        if child.tag == "sec":
            continue
        body_parts.append(extract_text_tokenizing_inline_figs(child))

    body_text = norm_ws(" ".join(p for p in body_parts if p))
    if body_text:
        chunks.append({"type": "body", "id": sid, "title": title, "text": body_text})

    for sub in sec.findall("sec"):
        chunks.extend(collect_sections(sub, parent_label=label))
    return chunks
def _abstract_text(abs_el: ET.Element) -> str:
    """Flatten a (possibly structured) <abstract> into plain text using your tokenizer."""
    parts = []

    def walk_abstract(el: ET.Element):
        # if structured abstract with <sec>, keep each sec title inline then its text
        if el.tag == "sec":
            t = el.find("title")
            if t is not None:
                parts.append(extract_text_tokenizing_inline_figs(t) + ": ")
            # body of this sec
            for child in list(el):
                if child.tag == "title":
                    continue
                walk_abstract(child)
            return

        # paragraphs or mixed content
        if el.text:
            parts.append(el.text)
        for child in list(el):
            walk_abstract(child)
        if el.tail:
            parts.append(el.tail)

    walk_abstract(abs_el)
    return norm_ws("".join(parts))

def extract_abstract_chunk(article: ET.Element) -> Optional[Dict]:
    """Return a single combined abstract chunk or None."""
    am = article.find("front/article-meta")
    if am is None:
        return None

    # collect candidate abstracts (exclude graphical/visual/teaser/toc)
    def is_narrative(a: ET.Element) -> bool:
        at = (a.get("abstract-type") or "").lower()
        return not any(k in at for k in ("graphical", "visual", "teaser", "toc"))

    abstracts = [a for a in am.findall("abstract") if is_narrative(a)]
    # optional: include translated abstracts the same way
    trans_abstracts = [a for a in am.findall("trans-abstract") if is_narrative(a)]

    if not abstracts and not trans_abstracts:
        return None

    texts = []
    for a in abstracts + trans_abstracts:
        # prefer an explicit title inside the abstract if present
        t_el = a.find("title")
        if t_el is not None:
            title_text = extract_text_tokenizing_inline_figs(t_el)
            if title_text:
                texts.append(title_text + ". ")
        texts.append(_abstract_text(a))

    all_text = norm_ws(" ".join(t for t in texts if t))
    if not all_text:
        return None

    # try to keep a stable id and title
    abs_id = (abstracts[0].get("id") if abstracts else None) or "abstract"
    abs_title = "Abstract"
    return {"type": "abstract", "id": abs_id, "title": abs_title, "text": all_text}

def jats_to_chunks(xml_string: str) -> List[Dict]:
    root = ET.fromstring(xml_string)
    article = root if root.tag == "article" else (root.find("article") or None)
    if article is None:
        raise ValueError("No <article> element found.")

    chunks: List[Dict] = []

    # --- front matter (if you have any, keep) ---

    # --- abstract (NEW) ---
    abs_chunk = extract_abstract_chunk(article)
    if abs_chunk is not None:
        chunks.append(abs_chunk)

    # --- body sections ---
    body = article.find("body")
    if body is not None:
        for sec in body.findall("sec"):
            chunks.extend(collect_sections(sec))

    # ---------- stand-alone figure placeholders (keep your code) ----------
    inline_fig_ids = set()
    if body is not None:
        for fig in body.findall(".//sec//fig"):
            fid = fig.get("id")
            if fid:
                inline_fig_ids.add(fid)

    def _add_fig_chunk(fig_el: ET.Element):
        tok = _fig_token(fig_el)
        chunks.append({
            "type": "body",
            "id": fig_el.get("id"),
            "title": (fig_el.findtext("label") or "").strip(),
            "text": tok
        })

    if body is not None:
        for fig in body.findall("fig"):
            fid = fig.get("id")
            if not fid or fid not in inline_fig_ids:
                _add_fig_chunk(fig)

    floats = article.find("floats-group")
    if floats is not None:
        for fig in floats.findall("fig"):
            fid = fig.get("id")
            if not fid or fid not in inline_fig_ids:
                _add_fig_chunk(fig)

    # --- back matter (keep your code) ---

    # normalize
    out = []
    for c in chunks:
        t = norm_ws(c.get("text", ""))
        if t:
            cc = dict(c); cc["text"] = t
            out.append(cc)
    return out

In [7]:
if __name__ == "__main__":
    # ----- helpers for preview -----
    def preview(chunks: List[Dict], n: int = 120):
        for i, ch in enumerate(chunks, 1):
            title = ch.get("title") or ch.get("section") or "(untitled)"
            print(f"[{i}] {ch['type']} :: {title}")
            print(ch["text"][:] + ("…" if len(ch["text"]) > n else ""))
            print("-" * 80)

    
    root = ET.fromstring(fulltext_xml)
    article = root if root.tag == "article" else first(root, "article")
    if article is None:
        raise ValueError("No <article> element found.")

    
    sections = jats_to_chunks(fulltext_xml)

    preview(sections)

[1] abstract :: Abstract
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoid isolated from Antrodia cinnamomea, has been reported to act against ALD, but its mechanisms of action are still not clear. In this study, we report for the first time the use of DEA (1) as a dual inhibitor of the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β in an in vitro ALD cell model. DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear translocation to activate downstream antioxidant genes. Importantly, DEA (1) restores the mitochondrial dysfunction induced by ethanol and generates antioxidant activity in the ALD cell model with minimal toxicity. We anticipate that DEA (1) could be a potential scaffold for the further development of clinical agen

In [8]:
# sections

# Zimanje link do tileshop na slikite

In [9]:
import re
import xml.etree.ElementTree as ET
from urllib.parse import urlencode

XLINK = "{http://www.w3.org/1999/xlink}"

WS_RE = re.compile(r"\s+")
def norm_ws(s: str) -> str:
    return WS_RE.sub(" ", (s or "")).strip()


SKIP_TAGS = {"ext-link", "uri", "inline-formula", "disp-formula", "table-wrap"}

def extract_text(node: ET.Element) -> str:
    parts = []
    def walk(el):
        if el.tag in SKIP_TAGS:
            if el.tail:
                parts.append(el.tail)
            return
        if el.text:
            parts.append(el.text)
        for child in list(el):
            walk(child)
        if el.tail:
            parts.append(el.tail)
    walk(node)
    return norm_ws("".join(parts))

def pmcid_number(pmcid: str) -> str:
    return pmcid.upper().replace("PMC", "")

def tileshop_inline_url(pmcid: str, filename: str, p: str = "PMC3",
                        title: str = "Click on image to zoom") -> str:
    q = {"title": title, "p": p, "id": f"{pmcid_number(pmcid)}_{filename}"}
    return "https://www.ncbi.nlm.nih.gov/core/lw/2.0/html/tileshop_pmc/tileshop_pmc_inline.html?" + urlencode(q)

def _first_graphic(fig: ET.Element) -> ET.Element | None:
    g = fig.find(".//graphic")
    if g is not None:
        return g
    alt = fig.find(".//alternatives/graphic")
    return alt

def figure_links_from_jats(xml_text: str, pmcid: str):
    """Return label, FULL caption text, and a best URL for each <fig>."""
    root = ET.fromstring(xml_text)
    article = root if root.tag == "article" else root.find("article")
    if article is None:
        return []

    out = []
    for fig in article.findall(".//fig"):
        label = (fig.findtext("label") or "").strip()

        
        cap_el = fig.find("caption")
        if cap_el is not None:
            caption = extract_text(cap_el)           
        else:
            caption = ""                             

       
        g = _first_graphic(fig)
        if g is None:
            continue
        href = g.get(f"{XLINK}href")
        if not href:
            continue

        if href.startswith(("http://", "https://", "ftp://")):
            url = href                 
        else:
            url = tileshop_inline_url(pmcid, href)  

        out.append({
            "label": label,
            "caption": caption,       
            "url": url,
            "href": href
        })
    return out


In [10]:
figs = figure_links_from_jats(fulltext_xml,pmcid)
figs

[{'label': 'Figure 1',
  'caption': 'Screening of compounds 1–13 and identification of DEA (1) as a potent Keap1–Nrf2 inhibitor in vitro with low toxicity. (A) The binding activity of Keap1–Nrf2 was measured via an FP assay with the compounds added at 50 µM. (B) Dose-response effect of compound DEA (1) on Keap1–Nrf2 binding. (C) Compound DEA (1) inhibited GSK3β activity in vitro. (D) Dose-response effect of compound DEA (1) on GSK3β activity. (E) Compound DEA (1) enhances ARE activity in LO2 cells determined by the dual luciferase assay. (F–H) The cytotoxicity of DEA (1) and ML334 on LO2, HEK 293T, and HepG2 cell lines for 48 h. Cells were treated with 0–100 µM of DEA (1) and ML334 for 48 h and cytotoxicity was detected using the MTT assay. Data are represented as mean ± SD. * p < 0.05, ** p < 0.01, *** p < 0.001 vs. DMSO group. Error bars represent the standard deviations of the results from three independent experiments. DEA: Dehydroeburicoic acid; FP: fluorescence polarization; DMSO

# Scrapeanje od tileshop stranata link do slikata i siminnjanje nejzino

In [11]:



def make_driver(headless: bool = True) -> webdriver.Chrome:
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--window-size=1280,1000")
    # be a tiny bit more like a real browser
    opts.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    return webdriver.Chrome(options=opts)


SEL_IMG = 'img[title="Click to change focus to this area of image."]'
FALLBACK_SEL = "img#tileshop, img.tileshop"

SAFE = re.compile(r"[^A-Za-z0-9._-]+")
def safe_name(s: str) -> str:
    return SAFE.sub("_", s).strip("_")

def resolve_img_src(driver: webdriver.Chrome, inline_url: str, timeout: int = 20) -> str:
    driver.get(inline_url)
    wait = WebDriverWait(driver, timeout)
    try:
        el = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, SEL_IMG)))
    except Exception:
        el = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, FALLBACK_SEL)))
    src = el.get_attribute("src") 
    return urljoin(inline_url, src)

def selenium_cookies_to_jar(driver: webdriver.Chrome, domain_hint: str) -> requests.cookies.RequestsCookieJar:
    jar = requests.cookies.RequestsCookieJar()
   
    for c in driver.get_cookies():
        dom = c.get("domain") or ""
        if domain_hint in dom:
            jar.set(c["name"], c["value"], domain=c.get("domain"), path=c.get("path", "/"))
    return jar

def looks_like_html(data: bytes) -> bool:
    head = data[:64].lstrip().lower()
    return head.startswith(b"<!doctype") or head.startswith(b"<html") or b"<body" in head

def ensure_image_bytes(raw: bytes) -> Image.Image:
    """Open with PIL to guarantee it's a real image."""
    im = Image.open(io.BytesIO(raw))
    im.load()
    return im

def pick_ext_from_content_type(ct: str) -> str:
    if not ct:
        return ".png"
    ct = ct.lower().split(";")[0].strip()
    return {
        "image/jpeg": ".jpg",
        "image/jpg": ".jpg",
        "image/png": ".png",
        "image/webp": ".webp",
        "image/tiff": ".tif",
        "image/gif": ".gif",
        "image/bmp": ".bmp",
    }.get(ct, ".png")

def download_image_with_headers(url: str, referer: str, cookies=None, timeout=60) -> (bytes, str):
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                       "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"),
        "Referer": referer,
        "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    }
    r = requests.get(url, headers=headers, cookies=cookies, timeout=timeout)
    r.raise_for_status()
    return r.content, r.headers.get("Content-Type", "")

def save_as_png_norm(raw: bytes, outpath: Path):
    """Normalize weird encodings to RGB PNG for consistency."""
    im = ensure_image_bytes(raw)
    if im.mode not in ("RGB", "RGBA"):
        im = im.convert("RGB")
    outpath.parent.mkdir(parents=True, exist_ok=True)
    im.save(outpath, format="PNG", optimize=True)


def download_all_figures(
    fig_list,
    pmcid: str,
    out_dir: str = r"D:\Users\User\VNPRagProekt\figures",
    headless: bool = True,
    retries: int = 3,           
    sleep: float = 1.0,
    figure_retries: int = 3      
):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    driver = make_driver(headless=headless)

    try:
        for fig in fig_list:
            label = fig.get("label", "(figure)")
            final_error = None

            for ftry in range(1, figure_retries + 1):
                try:
                    inline_url = fig.get("url") or fig.get("image_url")
                    if not inline_url:
                        raise RuntimeError("No inline tileshop URL present for this figure")

                   
                    img_url = resolve_img_src(driver, inline_url)
                    fig["image_url"] = img_url

                    
                    href = fig.get("href") or f"{label}.jpg"
                    base = safe_name(Path(href).stem)
                    fname = f"{pmcid}_{base}.png"
                    outpath = out_dir / fname

                    
                    cookies = selenium_cookies_to_jar(driver, domain_hint="ncbi.nlm.nih.gov")

                    
                    last_err = None
                    for attempt in range(1, retries + 1):
                        try:
                            raw, ctype = download_image_with_headers(
                                img_url, referer=inline_url, cookies=cookies, timeout=60
                            )
                            if (("image" not in (ctype or "").lower()) or looks_like_html(raw)):
                                raise RuntimeError(f"Non-image response (Content-Type={ctype!r})")
                            ensure_image_bytes(raw)  # validate
                            save_as_png_norm(raw, outpath)
                            fig["image_path"] = str(outpath.resolve())
                            print(f"✓ {label} → {outpath.name}")
                            last_err = None
                            break
                        except (requests.RequestException, UnidentifiedImageError, RuntimeError) as e:
                            last_err = e
                            if attempt < retries:
                                time.sleep(sleep * attempt)
                                continue
                            else:
                                raise

                   
                    final_error = None
                    break

                except Exception as e:
                    final_error = e
                    
                    if ftry < figure_retries:
                        time.sleep(sleep * ftry)
                        continue
                    else:
                       
                        fig.pop("image_path", None)
                        fig["error"] = str(final_error)
                        print(f"✗ {label} failed after {figure_retries} attempts: {final_error}")

    finally:
        driver.quit()

    return fig_list

In [12]:
results = download_all_figures(figs, pmcid, out_dir=r"D:\Users\User\VNPRagProekt\figures", headless=True)

✓ Figure 1 → PMC9866905_pharmaceuticals-16-00014-g001.png
✓ Figure 2 → PMC9866905_pharmaceuticals-16-00014-g002.png
✓ Figure 3 → PMC9866905_pharmaceuticals-16-00014-g003.png
✓ Figure 4 → PMC9866905_pharmaceuticals-16-00014-g004.png
✓ Figure 5 → PMC9866905_pharmaceuticals-16-00014-g005.png
✓ Figure 6 → PMC9866905_pharmaceuticals-16-00014-g006.png
✓ Figure 7 → PMC9866905_pharmaceuticals-16-00014-g007.png


In [18]:
figs

[{'label': 'Figure 1',
  'caption': 'Screening of compounds 1–13 and identification of DEA (1) as a potent Keap1–Nrf2 inhibitor in vitro with low toxicity. (A) The binding activity of Keap1–Nrf2 was measured via an FP assay with the compounds added at 50 µM. (B) Dose-response effect of compound DEA (1) on Keap1–Nrf2 binding. (C) Compound DEA (1) inhibited GSK3β activity in vitro. (D) Dose-response effect of compound DEA (1) on GSK3β activity. (E) Compound DEA (1) enhances ARE activity in LO2 cells determined by the dual luciferase assay. (F–H) The cytotoxicity of DEA (1) and ML334 on LO2, HEK 293T, and HepG2 cell lines for 48 h. Cells were treated with 0–100 µM of DEA (1) and ML334 for 48 h and cytotoxicity was detected using the MTT assay. Data are represented as mean ± SD. * p < 0.05, ** p < 0.01, *** p < 0.001 vs. DMSO group. Error bars represent the standard deviations of the results from three independent experiments. DEA: Dehydroeburicoic acid; FP: fluorescence polarization; DMSO

# Model za sliki

In [19]:
import base64, json, requests

OLLAMA_BASE = "http://127.0.0.1:11434"
MODEL = "qwen2.5vl:3b" 

SYSTEM_PROMPT = (
    "You are a bioscience figure explainer for a RAG pipeline. "
    "Write a summary that integrates evidence across all panels. "
    "Be objective and specific. Do not copy the caption. No citations. "
    "Return STRICT JSON only with the requested fields—no extra keys, no prose."
)

USER_TEMPLATE = (
    "Paper figure {LBL}. Use the official caption ONLY as context; do not copy it.\n\n"
    "Caption:\n{CAP}\n\n"
    "TASK: Synthesize all panels (A,B,...) into one short summary.\n\n"
    "Return JSON with EXACTLY these fields:\n"
    "{{\n"
    '  "summary": "≤120 words—main scientific takeaway across all panels",\n'
    '  "key_terms": ["term1", "term2", "..."],\n'
    '  "caveats": []\n'
    "}}\n"
    "Output JSON only."
)


def b64_image(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def describe_figure(fig: dict, temperature: float = 0.2, num_predict: int = 320) -> dict:
    user_prompt = USER_TEMPLATE.format(LBL=fig["label"], CAP=fig["caption"])
    img_b64 = b64_image(fig["image_path"])

    payload = {
        "model": MODEL,
        "stream": False,
        "options": {"temperature": temperature, "num_predict": num_predict},
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt, "images": [img_b64]},
        ],
    }

    r = requests.post(f"{OLLAMA_BASE}/api/chat", json=payload, timeout=300)
    try:
        r.raise_for_status()
    except requests.HTTPError as e:
        print("HTTP error from Ollama:", r.status_code, r.text[:500])
        raise e

    text = (r.json().get("message", {}) or {}).get("content", "").strip()


    try:
        resp = json.loads(text)
    except json.JSONDecodeError:
        return {"raw": text}


    resp.setdefault("summary", "")
    # resp.setdefault("rationale", "")
    resp.setdefault("key_terms", [])
    resp.setdefault("caveats", [])
    return resp


In [20]:
figure_descriptions = []
for fig in figs:
    out = {
        "label": fig["label"],
        "image_path": fig["image_path"],
        "caption": fig["caption"],
        "llm_description": describe_figure(fig),
    }
    figure_descriptions.append(out)


In [21]:
if len(figure_descriptions) != 0:
    figure_descriptions[0]['llm_description']

In [22]:
figure_descriptions

[{'label': 'Figure 1',
  'image_path': 'D:\\Users\\User\\VNPRagProekt\\figures\\PMC9866905_pharmaceuticals-16-00014-g001.png',
  'caption': 'Screening of compounds 1–13 and identification of DEA (1) as a potent Keap1–Nrf2 inhibitor in vitro with low toxicity. (A) The binding activity of Keap1–Nrf2 was measured via an FP assay with the compounds added at 50 µM. (B) Dose-response effect of compound DEA (1) on Keap1–Nrf2 binding. (C) Compound DEA (1) inhibited GSK3β activity in vitro. (D) Dose-response effect of compound DEA (1) on GSK3β activity. (E) Compound DEA (1) enhances ARE activity in LO2 cells determined by the dual luciferase assay. (F–H) The cytotoxicity of DEA (1) and ML334 on LO2, HEK 293T, and HepG2 cell lines for 48 h. Cells were treated with 0–100 µM of DEA (1) and ML334 for 48 h and cytotoxicity was detected using the MTT assay. Data are represented as mean ± SD. * p < 0.05, ** p < 0.01, *** p < 0.001 vs. DMSO group. Error bars represent the standard deviations of the res

# Spojutanje na dictionaries

In [23]:

desc_by_label = {d["label"]: d for d in figure_descriptions}


merged = {}

for entry in sections:
    title = entry.get("title")
    if title and title.startswith("Figure "):
        desc = desc_by_label.get(title)
        if desc:
            merged[title] = {
                **entry, 
                "label": desc["label"],
                "image_path": desc["image_path"],
                "caption": desc["caption"],
                "text": desc["llm_description"]["summary"], 
            }
        else:
            merged[title] = {**entry} 
    else:
        merged[entry.get("title", entry.get("id", ""))] = {**entry}



In [24]:
merged.keys()

dict_keys(['Abstract', '1. Introduction', '2.1. DEA (1) Inhibits the Keap1–Nrf2 PPI and GSK3β Activity', '2.2. DEA (1) Exhibits Low Cytotoxicity In Cellulo', '2.3. DEA (1) as a Dual Inhibitor of Keap1 and GSK3β', '2.4. DEA (1) Targets Keap1 and GSK3β to Induce Nrf2 Accumulation in the Nucleus and the Expression of Downstream Antioxidant Proteins', '2.5. DEA (1) Activates Nrf2 Downstream Antioxidant Genes in ALD Model Cells', '2.6. DEA (1) Repairs the EtOH Induced Mitochondrial Dysfunction and Improves the Antioxidant Activity in LO2 Cells', '3. Discussion', '4.1. Cell Lines and Culture', '4.2. Chemical', '4.3. Fluorescence Polarization Assay', '4.4. Western Blot (WB) and Co–Immunoprecipitation (Co–IP)', '4.5. Real-Time Quantitative Polymerase Chain Reaction (RT–qPCR)', '4.6. MTT Assay', '4.7. Detection of ROS and Antioxidant Activity', '4.8. Nuclear and Cytoplasmic Extraction', '4.9. GSK3β Kinase Assay', '4.10. Dual Luciferase Reporter Gene Assay', '4.11. Cellular Thermal Shift Assay (

In [25]:
# merged

In [26]:
for title, entry in merged.items():
    print("=" * 60)
    print(f"TITLE: {title}")
    print("-" * 60)
    
    print(entry.get("text"))
    

TITLE: Abstract
------------------------------------------------------------
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoid isolated from Antrodia cinnamomea, has been reported to act against ALD, but its mechanisms of action are still not clear. In this study, we report for the first time the use of DEA (1) as a dual inhibitor of the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β in an in vitro ALD cell model. DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear translocation to activate downstream antioxidant genes. Importantly, DEA (1) restores the mitochondrial dysfunction induced by ethanol and generates antioxidant activity in the ALD cell model with minimal toxicity. We anticipate that DEA (1) could be a potential s

In [27]:
# merged

# Semantic Chunking

In [28]:
emb = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
tokenizer = AutoTokenizer.from_pretrained("NeuML/pubmedbert-base-embeddings")


data = merged 


sections = []
for title, node in data.items():
    text = node.get("text", "")
   
    sections.append(
        Document(
            page_content=text.strip(),
            metadata={
                "section_title": title,
                "section_id": node.get("id"),
                "section_type": node.get("type"),
                "is_parent": True,      # marks the full-section parent
            },
        )
    )


text_splitter = SemanticChunker(emb)


all_chunks = []
for parent_doc in sections:
   
    sub_chunks = text_splitter.split_documents([parent_doc])

    
    for idx, ch in enumerate(sub_chunks):
        ch.metadata.update({
            "section_title": parent_doc.metadata["section_title"],
            "section_id": parent_doc.metadata["section_id"],
            "section_type": parent_doc.metadata["section_type"],
            
            "subchunk_index": idx,
           
        })
    all_chunks.extend(sub_chunks)


for i, d in enumerate(all_chunks):
    tokens = tokenizer.encode(d.page_content, truncation=False)
    print("=" * 60)
    print(f"Chunk {i+1} | section: {d.metadata.get('section_title')}"
          
          f" | sub_idx: {d.metadata.get('subchunk_index')}")
    print(f"Token count: {len(tokens)}")
    print(f"Character count: {len(d.page_content)}")
    
    preview = (d.page_content[:160] + "…") if len(d.page_content) > 160 else d.page_content
    print(preview)

C:\Users\User\AppData\Local\Temp\ipykernel_19152\2763678214.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")


Chunk 1 | section: Abstract | sub_idx: 0
Token count: 116
Character count: 500
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydr…
Chunk 2 | section: Abstract | sub_idx: 1
Token count: 93
Character count: 494
DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear tr…
Chunk 3 | section: 1. Introduction | sub_idx: 0
Token count: 745
Character count: 3761
Excessive alcohol consumption leads to oxidative stress in the liver as a result of alcohol metabolism, resulting in inflammatory damage and the injury of liver…
Chunk 4 | section: 1. Introduction | sub_idx: 1
Token count: 69
Character count: 298
Meanwhile, although a few GSK3β inhibitors have been reported for liver injury treatment, no GSK3β inhibitor to date has made it to the market [34]. Antrodia ci…
Chunk 5 | section: 1. I

# Normalizacija na chunks

In [33]:
from typing import List, Iterable, Tuple, Dict
import re
MIN_TOKENS = 120
MAX_TOKENS = 380
ALLOW_SENTENCE_OVERFLOW = True    
ABSORB_FIGURES_INTO_PREV = True    
NO_OVERLAP = True                  


SENT_BOUNDARY = re.compile(r'([.!?]["\')\]]*)\s+(?=\S)')

def sentences(text: str) -> List[str]:
    """
    Split text into whole sentences. Keeps punctuation and any trailing quotes/brackets
    with the sentence on the left. No lookbehinds => works in Python re.
    """
    parts = SENT_BOUNDARY.split(text.strip())
    if len(parts) == 1:
        return [parts[0]] if parts and parts[0].strip() else []

    out = []
    cur = parts[0]
    i = 1
    while i < len(parts):
        delim = parts[i]         
        nxt   = parts[i+1] if i+1 < len(parts) else ""
        cur = (cur + delim).strip()
        out.append(cur)
        cur = nxt
        i += 2

    if cur.strip():
        out.append(cur.strip())
    return out

def tokenize_len(text: str) -> int:
    return len(tokenizer.encode(text, truncation=False))

def pack_by_tokens(sents: List[str], max_tokens: int) -> List[str]:
    """
    Greedily pack whole sentences up to max_tokens.
    If one sentence alone > max_tokens:
      - keep it as its own (oversized) chunk if ALLOW_SENTENCE_OVERFLOW
      - else (not recommended) you could hard-slice here.
    """
    out, cur, cur_len = [], [], 0
    for s in sents:
        slen = tokenize_len(s)
        if slen > max_tokens and ALLOW_SENTENCE_OVERFLOW:
            if cur: out.append(" ".join(cur)); cur, cur_len = [], 0
            out.append(s)
            continue
        if not cur or cur_len + slen <= max_tokens:
            cur.append(s); cur_len += slen
        else:
            out.append(" ".join(cur)); cur, cur_len = [s], slen
    if cur:
        out.append(" ".join(cur))
    return out

def resegment_big_chunks(texts: List[str]) -> List[str]:
    """Re-pack any long raw chunks by sentence so none exceed MAX_TOKENS (except single long sentences)."""
    out = []
    for txt in texts:
        toks = tokenize_len(txt)
        if toks <= MAX_TOKENS:
            out.append(txt)
        else:
            out.extend(pack_by_tokens(sentences(txt), MAX_TOKENS))
    return out

def merge_small_chunks_within_section(texts: List[str]) -> List[str]:
    """Merge adjacent chunks within a section until each is >= MIN_TOKENS (except single long sentence case)."""
    if not texts: return []
    out, buf, buf_tok = [], "", 0
    for t in texts:
        t_tok = tokenize_len(t)
        if not buf:
            buf, buf_tok = t, t_tok
            continue
        
        if buf_tok < MIN_TOKENS and (buf_tok + t_tok) <= MAX_TOKENS:
            buf = (buf + " " + t).strip()
            buf_tok += t_tok
        else:
            out.append(buf)
            buf, buf_tok = t, t_tok
    if buf:
        out.append(buf)
    
    if len(out) >= 2 and tokenize_len(out[-1]) < MIN_TOKENS:
        if tokenize_len(out[-2]) + tokenize_len(out[-1]) <= MAX_TOKENS:
            out[-2] = (out[-2] + " " + out[-1]).strip()
            out.pop()
    return out

def is_figure_like(title: str) -> bool:
    t = (title or "").strip().lower()
    return t.startswith("figure") or t.startswith("table")


def normalize_chunks_by_section(docs: List[Document]) -> List[Document]:
    
    grouped: Dict[Tuple[str,str], List[Tuple[int,Document]]] = {}
    first_idx: Dict[Tuple[str,str], int] = {}

    for i, d in enumerate(docs):
        key = (d.metadata.get("section_id"), d.metadata.get("section_title"))
        grouped.setdefault(key, []).append((i, d))
        first_idx.setdefault(key, i)

   
    sections_sorted = sorted(grouped.items(), key=lambda kv: first_idx[kv[0]])

   
    absorbed: Dict[Tuple[str,str], List[Tuple[int,Document]]] = {}
    last_non_fig_key = None
    for key, items in sections_sorted:
        sec_id, sec_title = key
        if ABSORB_FIGURES_INTO_PREV and is_figure_like(sec_title) and last_non_fig_key is not None:
            absorbed.setdefault(last_non_fig_key, []).extend(items)
        else:
            absorbed.setdefault(key, []).extend(items)
            if not is_figure_like(sec_title):
                last_non_fig_key = key

   
    linear: List[Document] = []
    for key, items in sorted(absorbed.items(), key=lambda kv: first_idx[kv[0]]):
       
        items_sorted = sorted(
            items,
            key=lambda t: (
                t[1].metadata.get("subchunk_index") is None,
                t[1].metadata.get("subchunk_index", 10**9),
                t[0],
            )
        )
        base_meta = (items_sorted[0][1].metadata or {}).copy()
        texts = [it[1].page_content for it in items_sorted]
        texts = resegment_big_chunks(texts)
        texts = merge_small_chunks_within_section(texts)
       
        for t in texts:
            meta = {**base_meta, "is_parent": False}
            linear.append(Document(page_content=t, metadata=meta))

   
    compacted: List[Document] = []
    for d in linear:
        if compacted and tokenize_len(d.page_content) < MIN_TOKENS:
            prev = compacted[-1]
            if tokenize_len(prev.page_content) + tokenize_len(d.page_content) <= MAX_TOKENS:
               
                prev.page_content = (prev.page_content + " " + d.page_content).strip()
                
                pt = (prev.metadata.get("section_title") or "").strip()
                ct = (d.metadata.get("section_title") or "").strip()
                if pt and ct and pt != ct and not pt.endswith(ct) and f" + {ct}" not in pt:
                    prev.metadata["section_title"] = f"{pt} + {ct}"
                continue
        compacted.append(d)

   
    total = len(compacted)
    for i, d in enumerate(compacted):
        d.metadata["subchunk_index"] = i
        d.metadata["subchunk_count"] = total

    return compacted


normalized = normalize_chunks_by_section(all_chunks)


def report(docs: Iterable[Document]):
    for d in docs:
        n_tok = tokenize_len(d.page_content)
        print("="*60)
        print(f"{d.metadata.get('section_title')} | idx {d.metadata.get('subchunk_index')}/{d.metadata.get('subchunk_count')}")
        print(f"{n_tok} tokens | {len(d.page_content)} chars")
        print((d.page_content[:] + "…") if len(d.page_content) > 220 else d.page_content)

report(normalized)

Abstract | idx 0/28
207 tokens | 995 chars
Alcoholic liver disease (ALD) is a complicated disease which can lead to hepatocellular carcinoma; however, there is a lack of satisfactory therapeutics. Dehydroeburicoic acid (DEA) (1), a triterpenoid isolated from Antrodia cinnamomea, has been reported to act against ALD, but its mechanisms of action are still not clear. In this study, we report for the first time the use of DEA (1) as a dual inhibitor of the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β in an in vitro ALD cell model. DEA (1) engages Keap1 to disrupt the Keap1–Nrf2 PPI and inhibits GSK3β to restore Nrf2 activity in a Keap1-independent fashion. DEA (1) promotes Nrf2 nuclear translocation to activate downstream antioxidant genes. Importantly, DEA (1) restores the mitochondrial dysfunction induced by ethanol and generates antioxidant activity in the ALD cell model with minimal toxicity. We anticipate that DEA (1) could be a potential scaffold for the further developmen

# Embeding menager za embeding na chunks

In [34]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "NeuML/pubmedbert-base-embeddings"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: NeuML/pubmedbert-base-embeddings
Model loaded successfully. Embedding dimension: 768


# Vectordb

In [35]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "medical_papers", persist_directory: str = r'D:\Users\User\VectorDBZaRagVNP\data\vector_store'):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "Medical papers chunks embedding for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def empty_vector_store(self):
        
        self.client.delete_collection(self.collection_name)
       
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "Medical papers chunks embedding for RAG"}
        )
        print("Collection dropped and recreated. Count:", self.collection.count())

    def collection_count():
         print(f"Existing documents in collection: {self.collection.count()}")
        
    def add_documents(self, chunks: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            chunks: chunks of a medical paper 
            embeddings: Corresponding embeddings for the documents
        """
        if len(chunks) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(chunks)} documents to vector store...")
        
        
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
           
            chunk_id = f"chunk_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(chunk_id)
            
           
            metadata = dict()
           
            metadata['content_length'] = len(chunk)
            metadatas.append(metadata)
            
          
            documents_text.append(chunk)
            
          
            embeddings_list.append(embedding.tolist())
        
        
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(chunks)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise



In [36]:
vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: medical_papers
Existing documents in collection: 170


In [30]:
# vectorstore.empty_vector_store()

In [31]:
texts=[n.page_content for n in normalized]
embeddings=embedding_manager.generate_embeddings(texts)
vectorstore.add_documents(texts,embeddings)

Generating embeddings for 53 texts...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Generated embeddings with shape: (53, 768)
Adding 53 documents to vector store...
Successfully added 53 documents to vector store
Total documents in collection: 170


# RAG Retriever

In [37]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving paper chunks for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
       
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
           
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                papers = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, paper, metadata, distance) in enumerate(zip(ids, papers, metadatas, distances)):
                    retrieved_docs.append({
                        'id': doc_id,
                        'content': paper,
                        'metadata': metadata,
                        
                        'distance': distance,
                        'rank': i + 1
                    })
                
                print(f"Retrieved {len(retrieved_docs)} paper chunks (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []



In [38]:
rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [39]:
rag_retriever

In [40]:
rag_retriever.retrieve("How does DEA affect the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β activity in ALD model cells?")

Retrieving paper chunks for query: 'How does DEA affect the Keap1–Nrf2 protein–protein interaction (PPI) and GSK3β activity in ALD model cells?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)
Retrieved 5 paper chunks (after filtering)


[{'id': 'chunk_22b35c5c_25',
  'content': 'Recent studies on ALD are scarce compared to research on other liver diseases, and there are still no highly satisfactory therapeutic options for ALD. To our knowledge, no dual inhibitor of the Keap1–Nrf2 PPI and GSK3β to treat alcohol-induced liver injury has been reported. Compound DEA (1) potently inhibits Keap1-Nrf2 binding and GSK3β activity, enhancing ARE activity in vitro, and exhibits low cytotoxicity in various cell lines. Compound DEA (1) releases Nrf2 by binding with Keap1 and GSK3β, enhancing its thermal stability and reducing protein degradation. Compound DEA (1) regulates antioxidant effects by targeting Keap1 and GSK3β, as evidenced by decreased expression of antioxidant proteins in cells with knockdown of either Keap1 or GSK3β.',
  'metadata': {'content_length': 753},
  'distance': 104.04415893554688,
  'rank': 1},
 {'id': 'chunk_f92d20e1_8',
  'content': 'The ability of DEA (1) to disrupt the Keap1–Nrf2 binding in cellulo was 

In [41]:
from typing import Any, List
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage


try:
    from langchain_community.chat_models import ChatOllama
except ImportError:
    
    from langchain_ollama import ChatOllama


class LocalOllamaLLM:
    def __init__(
        self,
        model_name: str = "mistral:7b-instruct-v0.3-q4_0", # 32,768 tokens.
        base_url: str = "http://localhost:11434",
        temperature: float = 0.1,
        max_tokens: int = 1024,
        num_ctx: int = 8192,
        extra_model_kwargs: dict | None = None,
    ):
        """
        Initialize a local Ollama LLM.

        Args:
            model_name: Ollama model tag you've pulled locally.
            base_url: Ollama server URL.
            temperature: Sampling temperature.
            max_tokens: Max tokens to generate (mapped to num_predict).
            num_ctx: Context window to request from Ollama (if supported by model).
            extra_model_kwargs: Extra model kwargs to pass through to Ollama.
        """
        self.model_name = model_name
        self.base_url = base_url
        self.temperature = temperature
        self.max_tokens = max_tokens

        model_kwargs = {"num_ctx": num_ctx}
        if extra_model_kwargs:
            model_kwargs.update(extra_model_kwargs)

        
        self.llm = ChatOllama(
            model=self.model_name,
            temperature=self.temperature,
            base_url=self.base_url,
            
            num_predict=self.max_tokens,
            
            model_kwargs=model_kwargs,
        )

        print(f"Initialized local Ollama LLM with model: {self.model_name} at {self.base_url}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context (structured prompt).
        """
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template=(
                "You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.\n\n"
                "Context:\n{context}\n\n"
                "Question: {question}\n\n"
                "Answer: Provide a clear and informative answer based on the context above. "
                "If the context doesn't contain enough information to answer the question, say so."
            ),
        )
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            messages = [HumanMessage(content=formatted_prompt)]
            
            response = self.llm.invoke(messages)
            
            return response.content[:max_length]
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting.
        """
        simple_prompt = f"""Based on this context: {context}

        Question: {query}
        
        Answer:"""
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"

In [42]:
try:
    ollama_llm = LocalOllamaLLM()
    print("Local Ollama LLM initialized successfully!")
except Exception as e:
    print(f"Initialization error: {e}")
    ollama_llm = None

Initialized local Ollama LLM with model: mistral:7b-instruct-v0.3-q4_0 at http://localhost:11434
Local Ollama LLM initialized successfully!


C:\Users\User\AppData\Local\Temp\ipykernel_19152\3365943129.py:44: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(


In [43]:
from langchain.schema import HumanMessage

def rag_simple(
    query: str,
    retriever,
    llm,
    top_k: int = 3,
    return_debug: bool = False,
) -> str | Tuple[str, dict]:
    """
    Retrieve top_k chunks, build a concise prompt, and get an answer from the LLM.

    Works with:
      - LocalOllamaLLM wrapper you defined (has .generate_response and .llm.invoke)
      - Any LangChain chat model exposing .invoke([...])

    If return_debug=True, returns (answer, debug_info) where debug_info includes the
    used context and the raw retrieved docs.
    """
   
    results: List[Dict[str, Any]] = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc.get("content", "") for doc in results if doc.get("content")]) if results else ""
    if not context.strip():
        out = "No relevant context found to answer the question."
        return (out, {"context": "", "results": results}) if return_debug else out

    
    prompt = (
        "Use the following context to answer the question accurately and concisely.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\n\n"
        "Answer:"
    )

    
    try:
        if hasattr(llm, "generate_response"):
            
            answer = llm.generate_response(query=query, context=context)
        elif hasattr(llm, "invoke"):
            
            answer = llm.invoke([HumanMessage(content=prompt)]).content
        elif hasattr(llm, "llm") and hasattr(llm.llm, "invoke"):
            
            answer = llm.llm.invoke([HumanMessage(content=prompt)]).content
        else:
            raise TypeError("Unsupported llm object: expected a LangChain chat model or LocalOllamaLLM.")
    except Exception as e:
        answer = f"Error generating response: {e}"

    return (answer, {"context": context, "results": results}) if return_debug else answer

In [46]:
q = "What is dehydroeburicoic acid and from which natural source is it isolated?"
# q = "What is basketball?"
answer, dbg = rag_simple(q, rag_retriever, ollama_llm, top_k=5, return_debug=True)
print(answer)

Retrieving paper chunks for query: 'What is basketball?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)
Retrieved 5 paper chunks (after filtering)
 The provided context does not contain any information about basketball.
